# Model Klasifikasi Baju Layak dan Tidak Layak Menggunakan CNN Tanpa Pretraining

In [1]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define paths
train_dir = '/content/drive/MyDrive/Bangkit/data_split/train'
val_dir = '/content/drive/MyDrive/Bangkit/data_split/val'

## Preprocessing Data

In [5]:
def train_val_generators(train_dir, val_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary'
    )

    return train_generator, validation_generator


In [6]:
train_generator, validation_generator = train_val_generators(train_dir, val_dir)

Found 812 images belonging to 2 classes.
Found 374 images belonging to 2 classes.


## Membuat Model Tanpa Pretraining

In [18]:
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=1e-4)  # Set learning rate here
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


## Melatih Model

In [19]:
class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='val_accuracy', value=0.9, verbose=1):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            print(f"Early stopping requires {self.monitor} available!")
            return
        if current >= self.value:
            if self.verbose > 0:
                print(f"Epoch {epoch}: early stopping threshold reached - {self.monitor}: {current}")
            self.model.stop_training = True

In [20]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min'),
    EarlyStoppingByAccuracy(monitor='val_accuracy', value=0.9, verbose=1)
]

In [21]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=callbacks
)


Epoch 1/50
25/25 [==============================] - 644s 26s/step - loss: 1.0262 - accuracy: 0.6038 - val_loss: 0.8320 - val_accuracy: 0.4545 - lr: 1.0000e-04
Epoch 2/50
25/25 [==============================] - 19s 757ms/step - loss: 0.7133 - accuracy: 0.6615 - val_loss: 1.7373 - val_accuracy: 0.4631 - lr: 1.0000e-04
Epoch 3/50
25/25 [==============================] - 19s 742ms/step - loss: 0.6382 - accuracy: 0.6949 - val_loss: 1.7093 - val_accuracy: 0.4489 - lr: 1.0000e-04
Epoch 4/50
25/25 [==============================] - 18s 716ms/step - loss: 0.6887 - accuracy: 0.6846 - val_loss: 1.5450 - val_accuracy: 0.4602 - lr: 1.0000e-04
Epoch 5/50
25/25 [==============================] - 20s 784ms/step - loss: 0.7032 - accuracy: 0.6762 - val_loss: 0.8519 - val_accuracy: 0.4602 - lr: 1.0000e-04
Epoch 6/50
25/25 [==============================] - 19s 766ms/step - loss: 0.6266 - accuracy: 0.7115 - val_loss: 0.6627 - val_accuracy: 0.5966 - lr: 1.0000e-04
Epoch 7/50
25/25 [=======================

In [22]:
loss_model, accuracy_model = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f"Model Validation Accuracy: {accuracy_model * 100:.2f}%")

11/11 [==============================] - 2s 162ms/step - loss: 2.4193 - accuracy: 0.6420
Model Validation Accuracy: 64.20%
